<a href="https://colab.research.google.com/github/rcsb/py-rcsb-api/blob/master/notebooks/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RCSB PDB Data API: Quickstart

This Quickstart notebook will walk through the basics of creating and executing queries using this package. For more in-depth documentation, reference the [readthedocs page](https://rcsbapi.readthedocs.io/en/latest/index.html).

\
Before beginning, you must install the package:

```pip install rcsb-api```

In [ ]:
%pip install rcsb-api

## Creating and executing queries

To create a `Query` object, you need to provide three arguments:
- `input_type`: input_types are points where you can begin your query. Some examples are entry, polymer_entity, and polymer_entity_instance. For a full list of input_types see the [readthedocs](https://rcsbapi.readthedocs.io/en/latest/query_construction.html#input-type).
- `input_ids`: input_ids are accepted as either a dictionary (for single IDs) or a list of PDB-formatted IDs (for multiple IDs).
- `return_data_list`: list of data items to return. These must be unique path segments (specified using dots to separate each name). Further explained [below](#Providing-specific-and-unique-field-names/paths).

(More details on input arguments can be found in [readthedocs: Query Construction](https://rcsbapi.readthedocs.io/en/latest/query_construction.html#query-objects).)

For example, to create a `Query` object requesting all non-polymer components of a structure (ions, cofactors, etc):

In [30]:
from rcsbapi.data import Schema, Query
from pprint import pprint  # for easier-to-read output

query = Query(
    input_type="entry",
    input_ids=["4HHB"],
    return_data_list=["nonpolymer_bound_components"]  # must be unique field or unique path segment
)

After creating a `Query` object, you can now run it with `.exec()` or view the GraphQL query with `.get_editor_link()`:

In [35]:
# Execute the query and print the results
return_data = query.exec()
pprint(return_data)

{'data': {'entry': {'rcsb_entry_info': {'nonpolymer_bound_components': ['HEM']},
                    'rcsb_id': '4HHB'}}}


In [36]:
# Print the GraphQL editor URL
query.get_editor_link()

'https://data.rcsb.org/graphql/index.html?query=%7B%20entry%28entry_id%3A%20%224HHB%22%29%20%7B%0A%20%20rcsb_id%0A%20%20%20%20rcsb_entry_info%7B%0A%20%20%20%20%20%20nonpolymer_bound_components%0A%20%20%20%20%20%20%7D%0A%20%7D%0A%7D%0A'

### Querying multiple IDs
You can search multiple entries by starting from `input_type` "entries".

In [12]:
from rcsbapi.data import Schema, Query
from pprint import pprint  # for easier-to-read output

# Search multiple entries using "entries" instead of "entry"
query = Query(
    input_type="entries",
    input_ids=["4HHB", "12CA", "3PQR"],
    return_data_list=["nonpolymer_bound_components"]
)
return_data = query.exec()
pprint(return_data)

{'data': {'entries': [{'rcsb_entry_info': {'nonpolymer_bound_components': ['HEM']},
                       'rcsb_id': '4HHB'},
                      {'rcsb_entry_info': {'nonpolymer_bound_components': ['ZN']},
                       'rcsb_id': '12CA'},
                      {'rcsb_entry_info': {'nonpolymer_bound_components': ['NAG',
                                                                           'PLM',
                                                                           'RET']},
                       'rcsb_id': '3PQR'}]}}


### Querying multiple data items
You can also request multiple data items by adding to the `return_data_list`.

In [13]:
from rcsbapi.data import Schema, Query

#  Query multiple fields in return_data_list
query = Query(
    input_type="entries",
    input_ids=["4HHB", "12CA", "3PQR"],
    return_data_list=[
        "nonpolymer_bound_components",
        "citation.title",
        "rcsb_entry_info.polymer_composition"
    ]
)
return_data = query.exec()
pprint(return_data)

{'data': {'entries': [{'citation': [{'title': 'The crystal structure of human '
                                              'deoxyhaemoglobin at 1.74 A '
                                              'resolution'},
                                    {'title': 'Stereochemistry of Iron in '
                                              'Deoxyhaemoglobin'},
                                    {'title': 'Regulation of Oxygen Affinity '
                                              'of Hemoglobin. Influence of '
                                              'Structure of the Globin on the '
                                              'Heme Iron'},
                                    {'title': 'Three-Dimensional Fourier '
                                              'Synthesis of Human '
                                              'Deoxyhemoglobin at 2.5 '
                                              'Angstroms Resolution, I.X-Ray '
                                              'Ana

### Autocompletion of nested fields
If there are fields nested under a requested data item in `return_data_list`, the package will add all sub-fields to the query. This allows you to make more general requests to get all information under that field (e.g., `"exptl"`). If you would like a more precise query, you can still request specific fields (e.g., `"exptl.method"`).

In [14]:
from rcsbapi.data import Schema, Query

# Requesting "exptl" gets all fields underneath that field
query = Query(
    input_type="entries",
    input_ids=["4HHB"],
    return_data_list=["exptl"] # requests exptl.crystals_number, exptl.method, etc.
)
return_data = query.exec()
pprint(return_data)

{'data': {'entries': [{'exptl': [{'crystals_number': None,
                                  'details': None,
                                  'method': 'X-RAY DIFFRACTION',
                                  'method_details': None}],
                       'rcsb_id': '4HHB'}]}}


In [16]:
# To view the generated GraphQL query:
query.get_editor_link()

'https://data.rcsb.org/graphql/index.html?query=%7B%20entries%28entry_ids%3A%20%5B%224HHB%22%5D%29%20%7B%0A%20%20rcsb_id%0A%20%20%20%20exptl%7B%0A%20%20%20%20%20%20%20%20crystals_number%0A%20%20%20%20%20%20%20%20method%0A%20%20%20%20%20%20%20%20details%0A%20%20%20%20%20%20%20%20method_details%0A%20%20%20%20%20%20%7D%0A%20%7D%0A%7D%0A'

### Querying different `input_types`
You can also start queries from various `input_types` (e.g., `polymer_entities`, `polymer_entity_instances`, `uniprot`). (For more examples, see [readthedocs: Additional Examples](https://rcsbapi.readthedocs.io/en/latest/additional_examples.html))

In [ ]:
# Search from input_type "polymer_entities"
from rcsbapi.data import Schema, Query
query = Query(
    input_type="polymer_entities",
    input_ids=["2CPK_1", "3WHM_1", "2D5Z_1"],
    return_data_list=[
        "polymer_entities.rcsb_id",
        "rcsb_entity_source_organism.ncbi_taxonomy_id",
        "rcsb_entity_source_organism.ncbi_scientific_name",
        "cluster_id",
        "identity"
    ]
)
return_data = query.exec()
pprint(return_data)

In [ ]:
# Search from input_type "polymer_entity_instances"
from rcsbapi.data import Schema, Query
query = Query(
    input_type="polymer_entity_instances",
    input_ids=["4HHB.A", "12CA.A", "3PQR.A"],
    return_data_list=[
        "polymer_entity_instances.rcsb_id",
        "rcsb_polymer_instance_annotation.annotation_id",
        "rcsb_polymer_instance_annotation.name",
        "rcsb_polymer_instance_annotation.type"
    ]
)
return_data = query.exec()
pprint(return_data)

In [ ]:
# Search from input_type "uniprot"
from rcsbapi.data import Schema, Query
query = Query(
    input_type="uniprot",
    input_ids=["P68871"],
    return_data_list=[
        "rcsb_uniprot_annotation"
    ]
)
return_data = query.exec()
pprint(return_data)

## Determining fields for `return_data_list`

### Providing specific and unique field names/paths
There are some fields that must be further specified using multiple fields separated by dots. You can search for the dotation of a field by using `find_paths(input_type, field_name)`.

For example, the field, `"polymer_composition"`, is redundant between several nodes: 

In [ ]:
# The field "polymer_composition" isn't specific enough
from rcsbapi.data import Schema, Query
query = Query(
    input_type="entries",
    input_ids=["4HHB"],
    return_data_list=["polymer_composition"]
)

# This will throw a ValueError, which will print out up to 10 valid paths that you can use instead:

```
ValueError: "polymer_composition" exists, but is not a unique field, must specify further.
3 of 3 possible paths:
  assemblies.interfaces.rcsb_interface_info.polymer_composition
  assemblies.rcsb_assembly_info.polymer_composition
  rcsb_entry_info.polymer_composition
```

To get a list of all possible paths for a given field name (or a path segment, using dot notation, e.g., `field1.field2`), you can use the `Schema().find_paths()` method:
```python
from rcsbapi.data import Schema
schema = Schema()
schema.find_paths(input_type, field_name_or_path_segment)
```
For example:

In [21]:
# Find all paths:
from rcsbapi.data import Schema
schema = Schema()
schema.find_paths(input_type="entries", return_data_name="polymer_composition")

['assemblies.interfaces.rcsb_interface_info.polymer_composition',
 'assemblies.rcsb_assembly_info.polymer_composition',
 'rcsb_entry_info.polymer_composition']

In [23]:
# By looking through the list, find the intended field path
query = Query(
    input_type="entries",
    input_ids=["4HHB"],
    return_data_list=["rcsb_entry_info.polymer_composition"]
)
return_data = query.exec()
pprint(return_data)

{'data': {'entries': [{'rcsb_entry_info': {'polymer_composition': 'heteromeric '
                                                                  'protein'},
                       'rcsb_id': '4HHB'}]}}


###  Discovering field names
If you're unsure which fields exist, you can call `find_field_names(search_substring)`.

For example, to find all fields containing `"comp"`:

In [27]:
from rcsbapi.data import Schema, Query

schema = Schema()
schema.find_field_names("comp")

['chem_comps',
 'chem_comp',
 'label_comp_id',
 'chem_comp_monomers',
 'chem_comp_nstd_monomers',
 'pdbx_chem_comp_audit',
 'pdbx_chem_comp_descriptor',
 'pdbx_chem_comp_feature',
 'pdbx_chem_comp_identifier',
 'rcsb_chem_comp_annotation',
 'rcsb_chem_comp_container_identifiers',
 'rcsb_chem_comp_descriptor',
 'rcsb_chem_comp_info',
 'rcsb_chem_comp_related',
 'rcsb_chem_comp_synonyms',
 'rcsb_chem_comp_target',
 'mon_nstd_parent_comp_id',
 'pdbx_subcomponent_list',
 'comp_id',
 'component_id',
 'comp_id_1',
 'comp_id_2',
 'chem_comp_id',
 'compound_details',
 'subcomponent_ids',
 'rcsb_comp_model_provenance',
 'rcsb_branched_component_count',
 'beg_comp_id',
 'ligand_comp_id',
 'polymer_composition',
 'nonpolymer_comp',
 'nonpolymer_comp_id',
 'completeness',
 'target_comp_id',
 'pdb_format_compatible',
 'nonpolymer_bound_components',
 'cofactor_chem_comp_id']

Note that once you identify which field you want to use, you may need to also run the `find_paths()` method mentioned above on the field name to identify the set of possible field paths for `return_data_list`. 

In [28]:
# Find all paths for the field `"chem_comps"`:
from rcsbapi.data import Schema
schema = Schema()
schema.find_paths(input_type="entries", return_data_name="chem_comp")

['assemblies.branched_entity_instances.branched_entity.chem_comp_monomers.chem_comp',
 'assemblies.branched_entity_instances.branched_entity.prd.chem_comp',
 'assemblies.nonpolymer_entity_instances.nonpolymer_entity.nonpolymer_comp.chem_comp',
 'assemblies.nonpolymer_entity_instances.nonpolymer_entity.prd.chem_comp',
 'assemblies.polymer_entity_instances.polymer_entity.chem_comp_monomers.chem_comp',
 'assemblies.polymer_entity_instances.polymer_entity.chem_comp_nstd_monomers.chem_comp',
 'assemblies.polymer_entity_instances.polymer_entity.prd.chem_comp',
 'branched_entities.chem_comp_monomers.chem_comp',
 'branched_entities.prd.chem_comp',
 'nonpolymer_entities.nonpolymer_comp.chem_comp',
 'nonpolymer_entities.prd.chem_comp',
 'polymer_entities.chem_comp_monomers.chem_comp',
 'polymer_entities.chem_comp_nstd_monomers.chem_comp',
 'polymer_entities.prd.chem_comp']